In [ ]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import re
import operator
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import pydicom
from pymedphys_dicom.dicom import dicom_dose_interpolate

In [ ]:
energy = '06FFF'

In [ ]:
ROOT_DIR = Path(r"S:\Physics\RCCC Specific Files\Dosimetry\Film\MLCs\7SegA")

DICOM_DIR = (ROOT_DIR.joinpath(r"DICOM Exports")).joinpath(energy+"_7SegA")

RESULTS = ROOT_DIR.joinpath(r"Results")

In [ ]:
FILM_RESULTS_FILE = (RESULTS.joinpath(r"Data_7SegA.xlsx"))
film_results = pd.read_excel(FILM_RESULTS_FILE, sheet_name=energy+'_export').dropna()

In [ ]:
film_dose = film_results.Dose.values
film_x = film_results.x.values

In [ ]:
old_dosecheck_dose = pydicom.read_file(str(DICOM_DIR.joinpath('DoseCHECKCalculatedDose_20190724model\dose.dcm')), force=True)
new_dosecheck_dose = pydicom.read_file(str(DICOM_DIR.joinpath('DoseCHECKCalculatedDose\dose.dcm')), force=True)
monaco_dose = pydicom.read_file(str(DICOM_DIR.joinpath('RTDose\dose.dcm')), force=True)
dicom_plan = pydicom.read_file(str(DICOM_DIR.joinpath('RTPlan\plan.dcm')), force=True)

In [ ]:
interp_z = [0]
interp_y = [0]

interp_coords = (interp_z, interp_y, film_x)

In [ ]:
monaco_profile = np.squeeze(dicom_dose_interpolate(interp_coords, monaco_dose))
old_dosecheck_profile = np.squeeze(dicom_dose_interpolate(interp_coords, old_dosecheck_dose))
new_dosecheck_profile = np.squeeze(dicom_dose_interpolate(interp_coords, new_dosecheck_dose))

transmission_value = {
    '06MV': 100.56/100.12,
    '10MV': 100.87/100.11,
    '06FFF': 100.35/100.11,
    '10FFF': 100.32/100.13
}

In [ ]:
plt.figure(1)

plt.subplot(111)
plt.plot(film_x,monaco_profile,label='Monaco')
plt.plot(film_x,old_dosecheck_profile,label='DoseCHECK')
plt.plot(film_x,film_dose,label='Film')
plt.ylabel('Dose (Gy)')
plt.xlabel('Plane Position (mm)')
plt.title('')
plt.legend(loc='lower left')


plt.subplots_adjust(top=3, bottom=0.5, left=0, right=2, hspace=0.25,
                    wspace=0.25)

plt.suptitle(('MLC Test Fields, ' + energy + ' ' + '7SegA' + ' , doses for Monaco ' + ', 20190728 DoseCHECK, and film'),fontsize="x-large",x=1,y=3.2)

plt.savefig(RESULTS.joinpath('7SegA_' + energy + f'_20190728_model.png'),bbox_inches='tight')

In [ ]:
plt.figure(1)

plt.subplot(111)
plt.plot(film_x,monaco_profile,label='Monaco')
plt.plot(film_x,new_dosecheck_profile,label='DoseCHECK')
plt.plot(film_x,film_dose,label='Film')
plt.ylabel('Dose (Gy)')
plt.xlabel('Plane Position (mm)')
plt.title('')
plt.legend(loc='lower left')


plt.subplots_adjust(top=3, bottom=0.5, left=0, right=2, hspace=0.25,
                    wspace=0.25)

plt.suptitle(('MLC Test Fields, ' + energy + ' ' + '7SegA' + ' , doses for Monaco ' + ', 20190927 DoseCHECK, and film'),fontsize="x-large",x=1,y=3.2)

plt.savefig(RESULTS.joinpath('7SegA_' + energy + f'_20190927_model.png'),bbox_inches='tight')